# Imports

In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_excel("/content/hate_train.xlsx")
test_df = pd.read_excel("/content/hate_test.xlsx")
valid_df = pd.read_excel("/content/hate_valid.xlsx")


In [ ]:
!pip install pandas openpyxl mahaNLP indic-nlp-library fasttext transformers torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
print(train_df.head())
print(train_df.columns)
print(train_df['label'].value_counts())

                                                text label
0  @lokmat बहुसंख्यांक हिंदूंना घाबरट आणि मुर्ख ब...  HATE
1  @BalaNandgaonkar स्वतःचे खिसे भरत आहेत, यांना ...  HATE
2  या पुचाट UNO पेक्षा आमच्याकडचं ENO जास्त प्रभा...  HATE
3  एक हिजाब परिधान केलेली मुलगी काल परवापासून व्ह...  HATE
4  @KiritSomaiya @BJP4India @BJP4Maharashtra जंगी...  HATE
Index(['text', 'label'], dtype='object')
label
HATE    5375
OFFN    5375
PRFN    5375
NOT     5375
Name: count, dtype: int64


# PreProcessing

In [ ]:
!pip install pandas indic-nlp-library scikit-learn torch transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.2/481.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.11.1
    Uninstalling huggingface-hub-0.11.1:
      Successfully uninstalled huggingface-hub-0.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependenc

In [ ]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git


Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 20.42 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
import os
os.environ["INDIC_NLP_RESOURCES"] = "/content/indic_nlp_resources"


In [ ]:
!wget https://raw.githubusercontent.com/stopwords-iso/stopwords-mr/master/stopwords-mr.json


--2025-03-31 15:45:46--  https://raw.githubusercontent.com/stopwords-iso/stopwords-mr/master/stopwords-mr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1465 (1.4K) [text/plain]
Saving to: ‘stopwords-mr.json’

stopwords-mr.json   100%[===================>]   1.43K  --.-KB/s    in 0s      

2025-03-31 15:45:46 (29.5 MB/s) - ‘stopwords-mr.json’ saved [1465/1465]



In [ ]:
!wget -O mr.model https://github.com/anoopkunchukuttan/indic_nlp_resources/raw/master/morph/morfessor/mr.model


--2025-03-31 15:45:46--  https://github.com/anoopkunchukuttan/indic_nlp_resources/raw/master/morph/morfessor/mr.model
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/anoopkunchukuttan/indic_nlp_resources/master/morph/morfessor/mr.model [following]
--2025-03-31 15:45:47--  https://raw.githubusercontent.com/anoopkunchukuttan/indic_nlp_resources/master/morph/morfessor/mr.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10937694 (10M) [application/octet-stream]
Saving to: ‘mr.model’

mr.model            100%[===================>]  10.43M  --.-KB/s    in 0.07s   

2025-03-31 15:45:47 (150 MB/s) 

In [ ]:
import os

# Create the required directory structure
os.makedirs("morph/morfessor", exist_ok=True)

# Move the model file to the correct path
!mv mr.model morph/morfessor/mr.model


In [ ]:
!pip install stanza
import stanza
stanza.download("mr")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 27.6 MB/s eta 0:00:00


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: mr (Marathi) ...


INFO:stanza:Downloaded file to /root/stanza_resources/mr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [ ]:
marathi_stopwords = {
    "आहे", "आहेत", "मी", "आम्ही", "आमचा", "तुम्ही", "तू", "त्याला", "तिचा", "तो", "ती", "त्यांनी",
    "त्यांचा", "हे", "होता", "होते", "असा", "अशी", "असून", "असे", "की", "हे", "आणि", "परंतु",
    "मात्र", "जर", "तर", "किंवा", "पण", "हे", "सर्व", "काय", "कोण", "कधी", "का", "कसे", "असे",
    "नाही", "फार", "म्हणजे", "त्यामुळे", "म्हणून", "तेव्हा", "तसे", "कुठे", "सुद्धा", "अजून",
    "पण", "एक", "दोन", "तीन", "चार", "पाच", "होय", "तसेच", "यांना", "त्यांच्यासाठी", "म्हणजे",
    "होणार", "होत", "करा", "करून", "करायचे", "आणखीन", "कारण", "त्याच", "याचा", "याला"
}


In [ ]:
import pandas as pd
import re
import stanza

# Load Stanza NLP Pipeline for Marathi
nlp = stanza.Pipeline("mr", processors="tokenize,mwt,pos,lemma", use_gpu=True)

# Marathi Stopwords List
marathi_stopwords = {
    "आहे", "आहेत", "मी", "आम्ही", "आमचा", "तुम्ही", "तू", "त्याला", "तिचा", "तो", "ती", "त्यांनी",
    "त्यांचा", "हे", "होता", "होते", "असा", "अशी", "असून", "असे", "की", "आणि", "परंतु", "मात्र", "जर", "तर", "किंवा",
    "पण", "सर्व", "काय", "कोण", "कधी", "का", "कसे", "नाही", "फार", "म्हणजे", "त्यामुळे", "म्हणून", "तेव्हा", "तसे",
    "कुठे", "सुद्धा", "अजून", "एक", "दोन", "तीन", "चार", "पाच", "होय", "तसेच", "यांना", "त्यांच्यासाठी", "होणार",
    "होत", "करा", "करून", "करायचे", "आणखीन", "कारण", "त्याच", "याचा", "याला"
}

# Preprocessing Function
def preprocess_marathi_stanza(text):
    if pd.isna(text):  # Handle missing values
        return ""

    text = text.lower()  # Convert to lowercase
    text = re.sub(r'@\w+|http\S+|[^\w\s]|\d+', '', text)  # Remove special characters, URLs, numbers

    doc = nlp(text)  # Process text with Stanza

    # Tokenization, lemmatization, and stopword removal
    cleaned_text = " ".join([
        word.lemma for sent in doc.sentences for word in sent.words
        if word.lemma not in marathi_stopwords
    ])

    return cleaned_text



# Make copies to store cleaned text
train_cleaned = train_df.copy()
test_cleaned = test_df.copy()
valid_cleaned = valid_df.copy()

# Apply preprocessing
train_cleaned["clean_text"] = train_cleaned["text"].apply(preprocess_marathi_stanza)
test_cleaned["clean_text"] = test_cleaned["text"].apply(preprocess_marathi_stanza)
valid_cleaned["clean_text"] = valid_cleaned["text"].apply(preprocess_marathi_stanza)

# Save cleaned data for future use
train_cleaned.to_csv("/content/hate_train_clean.csv", index=False)
test_cleaned.to_csv("/content/hate_test_clean.csv", index=False)
valid_cleaned.to_csv("/content/hate_valid_clean.csv", index=False)

# Print first few rows to verify
print(train_cleaned[["text", "clean_text"]].head())


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: mr (Marathi):
| Processor | Package       |
-----------------------------
| tokenize  | ufal          |
| mwt       | ufal          |
| pos       | ufal_charlm   |
| lemma     | ufal_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


                                                text  \
0  @lokmat बहुसंख्यांक हिंदूंना घाबरट आणि मुर्ख ब...   
1  @BalaNandgaonkar स्वतःचे खिसे भरत आहेत, यांना ...   
2  या पुचाट UNO पेक्षा आमच्याकडचं ENO जास्त प्रभा...   
3  एक हिजाब परिधान केलेली मुलगी काल परवापासून व्ह...   
4  @KiritSomaiya @BJP4India @BJP4Maharashtra जंगी...   

                                          clean_text  
0                बहसखयक हदन घबरट आण मरख बनवण बद करणे  
1  सवत च खस भरत आहत यन समनय जनत मल तरणे कह फरक पड...  
2  य पचट uno पकष आमचयकडच eno जसत परभव आण गणकर असण...  
3  हजब परधन कलल मलग कल परवपसन वहयरल हतय courag e ...  
4  जग सवगत हणरच अनययचय वरधत भरणे जनत परटच परतयक क...  


# LSTM & ML models

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

# Only set the environment variable without trying to modify visible devices
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split

print("Using TensorFlow version:", tf.__version__)
print("Device settings:", tf.config.list_physical_devices())


train_df = pd.read_csv("/content/hate_train_clean.csv")
test_df = pd.read_csv("/content/hate_test_clean.csv")


print(f"Training data size: {train_df.shape}")
print(f"Testing data size: {test_df.shape}")


label_mapping = {"HATE": 0, "OFFN": 1, "PRFN": 2, "NOT": 3}
train_df["label"] = train_df["label"].map(label_mapping)
test_df["label"] = test_df["label"].map(label_mapping)



if len(train_df) > 5000:
    print(f"Reducing training dataset from {len(train_df)} to 5000 samples")
    train_df = train_df.sample(5000, random_state=42)

MAX_NUM_WORDS = 3000  # Very small vocabulary
MAX_SEQUENCE_LENGTH = 25  # Very short sequences

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["clean_text"])

X_train = tokenizer.texts_to_sequences(train_df["clean_text"])
X_test = tokenizer.texts_to_sequences(test_df["clean_text"])

X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

y_train = np.array(train_df["label"])
y_test = np.array(test_df["label"])


tf.keras.backend.clear_session()  # Clear any previous model

model = Sequential([
    Embedding(MAX_NUM_WORDS, 16),  # Tiny embedding size
    GlobalAveragePooling1D(),  # Simple pooling instead of recurrent layer
    Dense(8, activation='relu'),   # Minimal hidden layer
    Dropout(0.1),
    Dense(4, activation="softmax")  # 4 Output Classes
])


model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

print(model.summary())

# ✅ Train with smallest possible batch size
try:
    print("\nStarting training with extremely simplified settings...")
    history = model.fit(
        X_train, y_train,
        validation_split=0.1,
        epochs=2,           # Just 2 epochs
        batch_size=4,       # Tiny batch size
        verbose=1
    )


    model.save('marathi_toxic_comment_minimal_model.h5')


    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

except Exception as e:
    print("Error during training:", e)

    print("\nFalling back to traditional ML approach...")
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import LogisticRegression
    from sklearn.pipeline import Pipeline

    # Convert labels back to text for clarity in output
    reverse_mapping = {v: k for k, v in label_mapping.items()}
    train_df["text_label"] = train_df["label"].map(reverse_mapping)
    test_df["text_label"] = test_df["label"].map(reverse_mapping)

    # Create a simple ML pipeline
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=3000)),
        ('clf', LogisticRegression(max_iter=1000))
    ])

    # Train on original text
    pipeline.fit(train_df["clean_text"], train_df["label"])

    # Evaluate
    accuracy = pipeline.score(test_df["clean_text"], test_df["label"])
    print(f"Logistic Regression Accuracy: {accuracy * 100:.2f}%")

    def predict_text_sklearn(text):
        pred = pipeline.predict([text])[0]
        return reverse_mapping[pred]

    sample_text = "तुम्ही खूप चांगले काम केले आहे!"
    print(f"\nSample text: {sample_text}")
    print(f"Predicted (sklearn): {predict_text_sklearn(sample_text)}")


    import pickle
    with open('marathi_toxic_comment_sklearn_model.pkl', 'wb') as f:
        pickle.dump(pipeline, f)
    print("Saved sklearn model as marathi_toxic_comment_sklearn_model.pkl")

else:

    def predict_sample_text(sample_text):
        sample_seq = tokenizer.texts_to_sequences([sample_text])
        sample_padded = pad_sequences(sample_seq, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

        prediction = model.predict(sample_padded, verbose=0)
        predicted_class = np.argmax(prediction, axis=1)[0]

        reverse_label_mapping = {0: "HATE", 1: "OFFN", 2: "PRFN", 3: "NOT"}
        return reverse_label_mapping[predicted_class]

    # Example prediction
    sample_text = "तुम्ही खूप चांगले काम केले आहे!"
    print(f"\nSample text: {sample_text}")
    print(f"Predicted (neural network): {predict_sample_text(sample_text)}")

Using TensorFlow version: 2.18.0
Device settings: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training data size: (21500, 3)
Testing data size: (2000, 3)
Reducing training dataset from 21500 to 5000 samples


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None

Starting training with extremely simplified settings...
Epoch 1/2
Error during training: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
import pickle
import time

print("Starting Marathi toxic comment classification with sklearn...")


train_df = pd.read_csv("/content/hate_train_clean.csv")
test_df = pd.read_csv("/content/hate_test_clean.csv")

print(f"Training data size: {train_df.shape}")
print(f"Testing data size: {test_df.shape}")


label_mapping = {"HATE": 0, "OFFN": 1, "PRFN": 2, "NOT": 3}
reverse_mapping = {v: k for k, v in label_mapping.items()}

train_df["label"] = train_df["label"].map(label_mapping)
test_df["label"] = test_df["label"].map(label_mapping)


print("\nClass distribution in training data:")
print(train_df["label"].value_counts())


classifiers = {
    "LogisticRegression": LogisticRegression(max_iter=1000, C=1.0),
    "LinearSVC": LinearSVC(C=1.0, max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=100)
}


best_accuracy = 0
best_classifier = None
best_classifier_name = None

for name, classifier in classifiers.items():
    print(f"\n{'='*50}")
    print(f"Training {name}...")
    start_time = time.time()

    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=5000, ngram_range=(1, 2))),
        ('clf', classifier)
    ])

    # Train on original text
    pipeline.fit(train_df["clean_text"], train_df["label"])

    # Evaluate
    train_accuracy = pipeline.score(train_df["clean_text"], train_df["label"])
    test_accuracy = pipeline.score(test_df["clean_text"], test_df["label"])

    print(f"Training accuracy: {train_accuracy * 100:.2f}%")
    print(f"Testing accuracy: {test_accuracy * 100:.2f}%")
    print(f"Training time: {time.time() - start_time:.2f} seconds")

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_classifier = pipeline
        best_classifier_name = name

print(f"\n{'='*50}")
print(f"Best classifier: {best_classifier_name} with accuracy: {best_accuracy * 100:.2f}%")


model_filename = f'marathi_toxic_comment_{best_classifier_name.lower()}.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(best_classifier, f)
print(f"Saved best model as {model_filename}")


def predict_text(text):
    pred = best_classifier.predict([text])[0]
    return reverse_mapping[pred]

def predict_proba(text):
    if hasattr(best_classifier, 'predict_proba'):
        proba = best_classifier.predict_proba([text])[0]
        return {reverse_mapping[i]: round(p * 100, 2) for i, p in enumerate(proba)}
    else:
        # For models without predict_proba
        decision = best_classifier.decision_function([text])[0]
        # Convert to rough probabilities
        return {reverse_mapping[i]: max(0, min(100, round(d * 100, 2)))
                for i, d in enumerate(decision)}

# ✅ Example predictions with multiple samples
sample_texts = [
    "तुम्ही खूप चांगले काम केले आहे!",  # Positive example
    "मी तुमच्याशी सहमत नाही",  # Neutral example
    "तू एक मूर्ख आहेस"  # Potentially offensive example
]

print("\nSample predictions:")
for text in sample_texts:
    label = predict_text(text)
    print(f"Text: {text}")
    print(f"Predicted: {label}")

    # Show confidence scores
    print("Confidence scores:")
    probas = predict_proba(text)
    for label, score in probas.items():
        print(f"  {label}: {score}%")
    print()

Starting Marathi toxic comment classification with sklearn...
Training data size: (21500, 3)
Testing data size: (2000, 3)

Class distribution in training data:
label
0    5375
1    5375
2    5375
3    5375
Name: count, dtype: int64

Training LogisticRegression...
Training accuracy: 81.47%
Testing accuracy: 72.00%
Training time: 6.82 seconds

Training LinearSVC...
Training accuracy: 86.70%
Testing accuracy: 69.95%
Training time: 5.59 seconds

Training RandomForest...
Training accuracy: 99.40%
Testing accuracy: 69.40%
Training time: 41.95 seconds

Best classifier: LogisticRegression with accuracy: 72.00%
Saved best model as marathi_toxic_comment_logisticregression.pkl

Sample predictions:
Text: तुम्ही खूप चांगले काम केले आहे!
Predicted: NOT
Confidence scores:
  HATE: 17.48%
  OFFN: 27.03%
  PRFN: 15.89%
  NOT: 39.6%

Text: मी तुमच्याशी सहमत नाही
Predicted: NOT
Confidence scores:
  HATE: 13.76%
  OFFN: 24.77%
  PRFN: 23.02%
  NOT: 38.44%

Text: तू एक मूर्ख आहेस
Predicted: OFFN
Confidence 

# PreTrained model



In [ ]:
!pip install protobuf==3.20.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [ ]:
api_key= "743a8fb20087575b0dc88be695953c6896021ace"

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset as HFDataset
from sklearn.preprocessing import LabelEncoder

# Load Preprocessed Data
train_df = pd.read_csv("/content/hate_train_clean.csv")
test_df = pd.read_csv("/content/hate_test_clean.csv")

# Encode Labels
label_mapping = {"HATE": 0, "OFFN": 1, "PRFN": 2, "NOT": 3}
train_df["label"] = train_df["label"].map(label_mapping)
test_df["label"] = test_df["label"].map(label_mapping)

# Load MahaBERT or MahaHate Model
MODEL_NAME = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization
def tokenize_data(example):
    return tokenizer(example["clean_text"], padding="max_length", truncation=True, max_length=128)

# Convert Data to Hugging Face Dataset
train_data = HFDataset.from_dict({"clean_text": train_df["clean_text"], "label": train_df["label"]}).map(tokenize_data, batched=True)
test_data = HFDataset.from_dict({"clean_text": test_df["clean_text"], "label": test_df["label"]}).map(tokenize_data, batched=True)

# Load Model
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=4)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

# Train Model
trainer.train()

# Evaluate Model
trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/21500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'sop_classifier.classifier.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sahilphatangare55 (sahilphatangare55-dream-skrin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.251500,1.193918
2,1.087000,1.033205
3,0.927800,0.933705


{'eval_loss': 0.93370521068573,
 'eval_runtime': 14.0395,
 'eval_samples_per_second': 142.455,
 'eval_steps_per_second': 17.807,
 'epoch': 3.0}

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.912700,0.933705
2,0.889800,0.933705
3,0.888600,0.933705


TrainOutput(global_step=8064, training_loss=0.9087526703637744, metrics={'train_runtime': 1485.4398, 'train_samples_per_second': 43.421, 'train_steps_per_second': 5.429, 'total_flos': 385433077248000.0, 'train_loss': 0.9087526703637744, 'epoch': 3.0})

In [ ]:
from transformers import AutoTokenizer


save_path = "/content/maha_hate_model"

# Save the model
trainer.save_model(save_path)

if isinstance(tokenizer, AutoTokenizer):
    tokenizer.save_pretrained(save_path)
else:
    print("Error: Tokenizer is not an instance of AutoTokenizer. Re-initializing...")
    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
    tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved at: {save_path}")


Error: Tokenizer is not an instance of AutoTokenizer. Re-initializing...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model and tokenizer saved at: /content/maha_hate_model


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = "/content/maha_hate_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()

# Sample texts
sample_texts = [
    "तुम्ही खूप चांगले काम केले आहे!",  # Positive example
    "मी तुमच्याशी सहमत नाही",  # Neutral example
    "तू एक मूर्ख आहेस"  # Potentially offensive example
]


reverse_label_mapping = {0: "HATE", 1: "OFFN", 2: "PRFN", 3: "NOT"}


inputs = tokenizer(sample_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")

inputs = {key: value.to(device) for key, value in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

predicted_classes = torch.argmax(logits, dim=-1).cpu().tolist()


for text, pred_class in zip(sample_texts, predicted_classes):
    print(f"Sample Text: {text}")
    print(f"Predicted label: {reverse_label_mapping[pred_class]}\n")


Sample Text: तुम्ही खूप चांगले काम केले आहे!
Predicted label: NOT

Sample Text: मी तुमच्याशी सहमत नाही
Predicted label: HATE

Sample Text: तू एक मूर्ख आहेस
Predicted label: OFFN

